In [ ]:
import numpy as np
import pandas as pd

In [ ]:
labels = np.array(pd.read_csv("train_samples_updated.csv"))[:, 5:]
labels.shape
rgb_1_train = np.load("final_swin_rgb_train_view1.npy")
rgb_0_train = np.load("final_swin_rgb_train_view0.npy")
rgb_2_train = np.load("final_swin_rgb_train_view2.npy")
print(rgb_1_train.shape, rgb_0_train.shape, rgb_2_train.shape)

rgb_1_val = np.load("final_swin_rgb_val_view1.npy")
rgb_0_val = np.load("final_swin_rgb_val_view0.npy")
rgb_2_val = np.load("final_swin_rgb_val_view2.npy")
print(rgb_1_val.shape, rgb_0_val.shape, rgb_2_val.shape)

dct_1_train = np.load("swin_mul_dct_train_view1.npy")
dct_0_train = np.load("swin_mul_dct_train_view0.npy")
dct_2_train = np.load("swin_mul_dct_train_view2.npy")
print(dct_1_train.shape, dct_0_train.shape, dct_2_train.shape)

dct_1_val = np.load("swin_mul_dct_val_view1.npy")
dct_0_val = np.load("swin_mul_dct_val_view0.npy")
dct_2_val = np.load("swin_mul_dct_val_view2.npy")
print(dct_1_val.shape, dct_0_val.shape, dct_2_val.shape)

In [ ]:
val_labels = np.array(pd.read_csv("val_samples_updated.csv"))[:, 5:]

In [ ]:
train_transformer = np.load("train_transformer_10.npy")
val_transformer = np.load("val_transformer_10.npy")

In [ ]:
train_transformer.shape, val_transformer.shape

In [ ]:
import numpy as np
# import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.layers import merge
# from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Input, Dense, concatenate,multiply, LayerNormalization, Add
# from sklearn.linear_model import Ridge
from keras.layers import Lambda

In [ ]:
def split_tensor(X):
  s0, s1, s2 = tf.split(X, num_or_size_splits=3,axis = -1)
  return s0,s1,s2

def mul_sca(x):
    return tf.multiply(x[0],x[1])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
# seqLen = chunk_size-1
# seqLen = 14

#Middle Input LSTM
rgb_input_1 = Input(shape=(1024), name='input_rgb_1')
rgb_output_1 = Dense(64, activation='relu', name='output_rgb_1')(rgb_input_1)

#Middle Input LSTM
rgb_input_2 = Input(shape=(1024), name='input_rgb_2')
rgb_output_2 = Dense(64, activation='relu')(rgb_input_2)

#Right Input LSTM
rgb_input_3 = Input(shape=(1024), name='input_rgb_3')
rgb_output_3 = Dense(64, activation='relu')(rgb_input_3)


#Middle Input LSTM
dct_input_1 = Input(shape=(1024), name='input_dct_1')
dct_output_1 = Dense(64, activation='relu', name='output_dct_1')(dct_input_1)

#Middle Input LSTM
dct_input_2 = Input(shape=(1024), name='input_dct_2')
dct_output_2 = Dense(64, activation='relu')(dct_input_2)

#Right Input LSTM
dct_input_3 = Input(shape=(1024), name='input_dct_3')
dct_output_3 = Dense(64, activation='relu')(dct_input_3)




# layer = tf.keras.layers.LayerNormalization(axis=-1)
rgb_1_nor_output = LayerNormalization()(rgb_output_1)
rgb_2_nor_output = LayerNormalization()(rgb_output_2)
rgb_3_nor_output = LayerNormalization()(rgb_output_3)


dct_1_nor_output = LayerNormalization()(dct_output_1)
dct_2_nor_output = LayerNormalization()(dct_output_2)
dct_3_nor_output = LayerNormalization()(dct_output_3)


#Merged Layer
merged_rgb = concatenate([rgb_output_1, rgb_output_2, rgb_output_3], name='RGB_Concatenate')
merge_dense_rgb = Dense(64, activation='relu')(merged_rgb)
softmax_output_rgb = Dense(3, activation='softmax', name='soft_score_rgb')(merge_dense_rgb)
weight_rgb1, weight_rgb2,weight_rgb3 = Lambda(split_tensor)(softmax_output_rgb)

score_rgb1 = Lambda(mul_sca)([rgb_1_nor_output, weight_rgb1])
score_rgb2 = Lambda(mul_sca)([rgb_2_nor_output, weight_rgb2])
score_rgb3 = Lambda(mul_sca)([rgb_3_nor_output, weight_rgb3])
additive_score_rgb = Add()([score_rgb1, score_rgb2, score_rgb3])



merged_dct = concatenate([dct_output_1, dct_output_2, dct_output_3], name='DCT_Concatenate')
merge_dense_dct = Dense(64, activation='relu')(merged_dct)
softmax_output_dct = Dense(3, activation='softmax', name='soft_score_dct')(merge_dense_dct)
weight_dct1, weight_dct2,weight_dct3 = Lambda(split_tensor)(softmax_output_dct)

score_dct1 = Lambda(mul_sca)([dct_1_nor_output, weight_dct1])
score_dct2 = Lambda(mul_sca)([dct_2_nor_output, weight_dct2])
score_dct3 = Lambda(mul_sca)([dct_3_nor_output, weight_dct3])
additive_score_dct = Add()([score_dct1, score_dct2, score_dct3])


# transformer_features = Input(shape=(768), name='transformer')
# trans_output= Dense(128, activation='relu', name='output_trans_1')(transformer_features)
# tran_nor_output = LayerNormalization()(trans_output)



# rgb_dct_concat = concatenate([additive_score_rgb, additive_score_dct, tran_nor_output], name='rgb_dct_Concatenate')
rgb_dct_concat = Add()([additive_score_rgb, additive_score_dct])

# final_model_output11 = Dense(256, activation='relu')(rgb_dct_concat)
final_model_output12 = Dense(64, activation='relu')(rgb_dct_concat)
final_model_output = Dense(14, activation='sigmoid')(final_model_output12)

final_model = Model(inputs=[rgb_input_1, rgb_input_2, rgb_input_3, dct_input_1, dct_input_2, dct_input_3], outputs=final_model_output,name='Final_output')

# opt = keras.optimizers.Adam(learning_rate=0.01) #Optimizer
# final_model.compile(optimizer = opt, loss = 'binary_crossentropy')

In [ ]:
final_model.summary()

In [ ]:

opt = keras.optimizers.SGD(learning_rate=0.01) #Optimizer
final_model.compile(optimizer = opt, loss = 'binary_crossentropy')


In [ ]:
zero_bias_history = final_model.fit([rgb_1_train, rgb_0_train, rgb_2_train, dct_1_train, dct_0_train, dct_2_train], labels, epochs = 50, batch_size = 32, validation_split = 0.1, callbacks = [callback])

# callbacks = [callback]


In [ ]:
predict = final_model.predict([rgb_1_val, rgb_0_val, rgb_2_val, dct_1_val, dct_0_val, dct_2_val])

predict

In [ ]:
train_csv_file = pd.read_csv("val_samples_updated.csv")

col_names = ['rec_no', 'subject_pos', 'start_time', 'end_time' ]
new_train_csv = train_csv_file.drop(col_names, axis=1)

#preparing train and test csv
test_csv = new_train_csv
# train_csv = new_train_csv[3123:]
print(len(test_csv))

Column_names = ['Settle','Legs crossed','Groom','Hand-mouth','Fold arms','Leg movement','Scratch','Gesture','Hand-face','Adjusting clothing','Fumble','Shrug','Stretching','Smearing hands']
# Column_names = ['Hand-face','Hand-mouth','Gesture','Fumble','Scratch','Stretching','Smearing hands','Shrug','Adjusting clothing','Groom','Fold arms','Leg movement','Settle','Legs crossed']
extracted_col = test_csv["sample_id"]
test_pred_csv = pd.DataFrame(predict, columns = Column_names)
test_pred_csv.insert(0, "sample_id", extracted_col)
# test_pred_csv.to_csv("test_predicted_dct_swin_fusion_"  + ".csv", index=False)

In [ ]:
import pandas as pd, numpy as np
from sklearn.metrics import average_precision_score


CLASSES = ['Hand-face','Hand-mouth','Gesture','Fumble','Scratch','Stretching','Smearing hands','Shrug','Adjusting clothing','Groom','Fold arms','Leg movement','Settle','Legs crossed']


def evaluate(test_annotation_file,user_submission_file):
#     test = pd.read_csv(test_annotation_file,index_col="sample_id").sort_values('sample_id')
#     user = pd.read_csv(user_submission_file,index_col="sample_id").sort_values('sample_id')
    
    test = test_annotation_file.sort_values('sample_id')
    
    user = user_submission_file.sort_values('sample_id')
    if not(np.all(test.index==user.index)):
        raise ValueError("Indexes of test and prediction files do not agree.")
        
    scores = []
    for behaviour in CLASSES:
        cur_score = average_precision_score(test[behaviour].values,user[behaviour].values)
        scores.append(cur_score)
    per_class_scores = pd.DataFrame({'behaviour':CLASSES,'score':scores}).set_index('behaviour')
    macro_average = np.mean(scores)
    return {'macro_average':macro_average,'per_class_scores':per_class_scores}



if __name__=='__main__':
    # example usage of evaluate function
    test_annotation_file = test_csv
    user_submission_file = test_pred_csv # use your own predictions here
    results = evaluate(test_annotation_file,user_submission_file)
    print('')
    print('--------------- MACRO AVERAGE: -----------------')
    print('')
    print(str(results['macro_average']))
    print('')
    print('--------------- PER CLASS: ---------------------')
    print(str(results['per_class_scores']))


In [ ]:
intermediate_model_soft= Model(inputs=final_model.input ,outputs=final_model.get_layer(index = 16).output)
# intermediate_model_soft.summary()
rgb_weights = intermediate_model_soft.predict([rgb_1_val, rgb_0_val, rgb_2_val, dct_1_val, dct_0_val, dct_2_val])

intermediate_model_soft= Model(inputs=final_model.input ,outputs=final_model.get_layer(index = 17).output)
# intermediate_model_soft.summary()
dct_weights = intermediate_model_soft.predict([rgb_1_val, rgb_0_val, rgb_2_val, dct_1_val, dct_0_val, dct_2_val])

In [ ]:
np.mean(rgb_weights, axis = 0), np.mean(dct_weights, axis = 0)

In [ ]:
pd.DataFrame(dct_weights).to_csv("RGB_Weights.csv")

In [ ]:
# final_model.save("rgb_dct_fusion_model_0.479")

In [ ]:
test_transformer = np.load("test_transformer_10.npy")

rgb_1_test = np.load("final_swin_rgb_test_view1.npy")
rgb_0_test = np.load("final_swin_rgb_test_view0.npy")
rgb_2_test = np.load("final_swin_rgb_test_view2.npy")

dct_1_test = np.load("swin_mul_dct_test_view1.npy")
dct_0_test = np.load("swin_mul_dct_test_view0.npy")
dct_2_test = np.load("swin_mul_dct_test_view2.npy")

print(test_transformer.shape)
print(rgb_1_test.shape, rgb_0_test.shape, rgb_2_test.shape, dct_1_test.shape, dct_0_test.shape, dct_2_test.shape )

In [ ]:
test_predict = final_model.predict([rgb_1_test, rgb_0_test, rgb_2_test, dct_1_test, dct_0_test, dct_2_test])

test_predict

In [ ]:
test_fake_csv_file = pd.read_csv("fake_predictions_test.csv")


# train_csv = new_train_csv[3123:]
print(len(test_fake_csv_file))

Column_names = ['Settle','Legs crossed','Groom','Hand-mouth','Fold arms','Leg movement','Scratch','Gesture','Hand-face','Adjusting clothing','Fumble','Shrug','Stretching','Smearing hands']
# Column_names = ['Hand-face','Hand-mouth','Gesture','Fumble','Scratch','Stretching','Smearing hands','Shrug','Adjusting clothing','Groom','Fold arms','Leg movement','Settle','Legs crossed']
extracted_col = test_fake_csv_file["sample_id"]
test_pred_csv = pd.DataFrame(test_predict, columns = Column_names)
test_pred_csv.insert(0, "sample_id", extracted_col)
# test_pred_csv.to_csv("test_predicted_fusion_version_11"  + ".csv", index=False)

In [ ]:
len(test_fake_csv_file), len(test_pred_csv)

In [ ]:
test_pred_csv
test_pred_csv.to_csv("test4"  + ".csv", index=False)

In [ ]:
test_fake_csv_file.head()

In [ ]:
import pandas as pd, numpy as np
from sklearn.metrics import average_precision_score


CLASSES = ['Hand-face','Hand-mouth','Gesture','Fumble','Scratch','Stretching','Smearing hands','Shrug','Adjusting clothing','Groom','Fold arms','Leg movement','Settle','Legs crossed']


def evaluate(test_annotation_file,user_submission_file):
#     test = pd.read_csv(test_annotation_file,index_col="sample_id").sort_values('sample_id')
#     user = pd.read_csv(user_submission_file,index_col="sample_id").sort_values('sample_id')
    
    test = test_annotation_file.sort_values('sample_id')
    
    user = user_submission_file.sort_values('sample_id')
    if not(np.all(test.index==user.index)):
        raise ValueError("Indexes of test and prediction files do not agree.")
        
    scores = []
    for behaviour in CLASSES:
        cur_score = average_precision_score(test[behaviour].values,user[behaviour].values)
        scores.append(cur_score)
    per_class_scores = pd.DataFrame({'behaviour':CLASSES,'score':scores}).set_index('behaviour')
    macro_average = np.mean(scores)
    return {'macro_average':macro_average,'per_class_scores':per_class_scores}



if __name__=='__main__':
    # example usage of evaluate function
    test_annotation_file = test_fake_csv_file
    user_submission_file = test_pred_csv # use your own predictions here
    results = evaluate(test_annotation_file,user_submission_file)
    print('')
    print('--------------- MACRO AVERAGE: -----------------')
    print('')
    print(str(results['macro_average']))
    print('')
    print('--------------- PER CLASS: ---------------------')
    print(str(results['per_class_scores']))


In [ ]:
final_model.save("additive_rgb_dct_transformer_50_0.472")

In [ ]:
from keras.utils import plot_model
plot_model(final_model, to_file='model12.png')